In [1]:
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import shap

In [3]:
df = pd.read_csv('seasonal_dataset.csv')

In [4]:
df.shape

(3236, 111)

# Feature cleaning

### drop all one value feature 

In [5]:
for col in df.columns:
    if df[col].value_counts().count() == 1:
        df.drop([col], axis=1, inplace=True)

### drop anomalies in "Sitlav - Soybeans - Yield - KG/Ha"

In [6]:
drop_anomalies_list = df[df['Sitlav - Soybeans - Yield - KG/Ha'] >= 10000]
df = df.drop([987, 1451], axis=0)

### Drop row if "plot code" is null

In [7]:
df['plot code'].isnull().value_counts()

False    3218
True       16
Name: plot code, dtype: int64

In [8]:
df = df[df['plot code'].notna()]

In [9]:
df['plot code'].isnull().value_counts()

False    3218
Name: plot code, dtype: int64

In [10]:
df['Season code'].isnull().value_counts()

False    3218
Name: Season code, dtype: int64

## Feature Engineering

### The categorical variables prep

In [11]:
categorical = [var for var in df.columns if df[var].dtype=='O']
print('There are {} categorical variables'.format(len(categorical)))
print('The categorical variables are :', categorical)

There are 3 categorical variables
The categorical variables are : ['plot code', 'Season code', 'CLASSE_DOM']


In [12]:
def one_hot_encoding(pandas_series):    
    # Apply one-hot encoding
    one_hot = pd.get_dummies(pandas_series)
    return(one_hot)

#### CLASSE_DOM

In [13]:
df['CLASSE_DOM'].value_counts()

LVAd    1322
LVd      750
RQo      675
FFc      183
PVAd     152
RLd      107
LAd       23
PVe        6
Name: CLASSE_DOM, dtype: int64

In [14]:
df.shape

(3218, 110)

In [15]:
df = pd.concat([df, one_hot_encoding(df['CLASSE_DOM'])], axis=1)
df = df.drop(['CLASSE_DOM'], axis=1)

In [16]:
df.shape

(3218, 117)

#### plot code

In [17]:
from sklearn.preprocessing import LabelEncoder
# Label encoding: In this method, each category is assigned a unique numerical value. 
def Label_Encoder(pandas_series):
    le = LabelEncoder()
    label_encoded = le.fit_transform(pandas_series)
    return(label_encoded)

In [18]:
df['plot code'].value_counts()

AL549648Z    3
AL549261Z    3
AL549294Z    3
AL549290Z    3
AL549284Z    3
            ..
AL549751Z    1
AL549752Z    1
AL551731Z    1
AL549753Z    1
AL551419Z    1
Name: plot code, Length: 1378, dtype: int64

In [19]:
df['plot code'] = Label_Encoder(df['plot code'])

In [20]:
df['plot code'].value_counts()

919     3
688     3
719     3
715     3
709     3
       ..
929     1
930     1
1263    1
931     1
1146    1
Name: plot code, Length: 1378, dtype: int64

In [21]:
df['Sitlav - Soybeans - Yield - KG/Ha'].isnull().sum()

0

#### Fill missing values

In [22]:
for col in df.columns:
    s = df[col].value_counts(normalize=True)
    missing = df[col].isnull()
    df.loc[missing,col] = np.random.choice(s.index, size=len(df[missing]),p=s.values)

In [23]:
df.isnull().sum()

plot code                            0
Season code                          0
Sitlav - Soybeans - Yield - KG/Ha    0
mean_savi2                           0
min_savi2                            0
                                    ..
LVd                                  0
PVAd                                 0
PVe                                  0
RLd                                  0
RQo                                  0
Length: 117, dtype: int64

In [24]:
df.describe()

,plot code,Sitlav - Soybeans - Yield - KG/Ha,mean_savi2,min_savi2,max_savi2,skew_savi2,kurt_savi2,std_savi2,mean_ndwi,min_ndwi,...,skew_max temp,kurt_max temp,FFc,LAd,LVAd,LVd,PVAd,PVe,RLd,RQo
count,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,...,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000
mean,610.937539,3576.861467,0.028669,0.019006,0.037503,-0.154753,-1.005721,0.009081,-0.624618,-0.780838,...,-0.172593,-0.410390,0.056868,0.007147,0.410814,0.233064,0.047234,0.001865,0.033250,0.209758
std,365.251800,660.466253,0.004676,0.005630,0.005707,1.031666,2.642147,0.003466,0.071646,0.069515,...,0.353708,0.414356,0.231625,0.084252,0.492058,0.422848,0.212173,0.043146,0.179318,0.407199
min,0.000000,285.900000,0.008726,0.002392,0.012534,-2.125023,-5.999652,0.000051,-0.837559,-0.880751,...,-1.036083,-1.224658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,294.250000,3235.500000,0.025604,0.015975,0.034615,-0.975815,-2.849101,0.006964,-0.669787,-0.826073,...,-0.445837,-0.682809,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,595.000000,3600.000000,0.028527,0.017814,0.038156,-0.138377,-1.422820,0.009405,-0.625887,-0.796701,...,-0.180206,-0.468072,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,895.750000,3965.950000,0.031523,0.020183,0.041093,0.549419,1.228256,0.011456,-0.583248,-0.758747,...,0.108718,-0.193907,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1377.000000,10960.300000,0.046577,0.046411,0.053486,1.997825,4.592682,0.024836,-0.304926,-0.304926,...,0.773722,2.037502,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
df.groupby(['plot code', 'Season code', 'Sitlav - Soybeans - Yield - KG/Ha']).count().reset_index()

,plot code,Season code,Sitlav - Soybeans - Yield - KG/Ha,mean_savi2,min_savi2,max_savi2,skew_savi2,kurt_savi2,std_savi2,mean_ndwi,...,skew_max temp,kurt_max temp,FFc,LAd,LVAd,LVd,PVAd,PVe,RLd,RQo
0,0,CR196IKKIJ5Z,3789.8,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,0,CR196LWQ0DXZ,4666.8,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,0,CR196PUJKZ,2451.5,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,CR196J6Q05CZ,4090.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,CR196P7HHZ,3930.6,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,1373,CR196NRQ0NHZ,3386.2,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3214,1374,CR196ILEIG8Z,4283.5,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3215,1375,CR196PGQ046Z,4705.7,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3216,1376,CR196LHPSZ,3365.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


# Data prep

### Feature selection

In [26]:
y = df['Sitlav - Soybeans - Yield - KG/Ha'].copy()
X = df.drop(['Sitlav - Soybeans - Yield - KG/Ha', 'Season code', 'plot code'], axis=1)

In [27]:
X.shape, y.shape

((3218, 114), (3218,))

In [28]:
def feature_selection(estimator, X, y):
    selector = SelectFromModel(estimator=estimator).fit(X, y)
    filter_list = selector.get_support()
    filtered_list = [x for x, condition in zip(X.columns, filter_list) if condition]
    return (filtered_list)

In [29]:
filtered_list = feature_selection(LinearRegression(), X, y)
# filtered_list = feature_selection(RidgeCV(), X, y)
# filtered_list = feature_selection(LassoCV(), X, y)

In [30]:
print("Number of features after selection:", len(filtered_list))

Number of features after selection: 10


In [31]:
X = X[filtered_list]

### Scaling features to a range

In [32]:
min_max_scaler = MinMaxScaler()
max_abs_scaler = MaxAbsScaler()

In [33]:
def Normalization(scaler, X):
    X_train = scaler.fit_transform(X)
    X_test = scaler.fit_transform(X)
    return(X)

In [34]:
X = Normalization(max_abs_scaler, X)

### Train test split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [36]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2896, 10), (322, 10), (2896,), (322,))

### Building the model

# Linear Regression model

In [37]:
# Linear Regression model
# Assigning the algorithm to the variable
lr = LinearRegression()

# Fitting of the model
lr.fit(X_train, y_train)

LinearRegression()

In [38]:
lr_y_train_pred = lr.predict(X_train)
lr_y_test_pred = lr.predict(X_test)

In [41]:
df.describe()

,plot code,Sitlav - Soybeans - Yield - KG/Ha,mean_savi2,min_savi2,max_savi2,skew_savi2,kurt_savi2,std_savi2,mean_ndwi,min_ndwi,...,skew_max temp,kurt_max temp,FFc,LAd,LVAd,LVd,PVAd,PVe,RLd,RQo
count,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,...,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000
mean,610.937539,3576.861467,0.028669,0.019006,0.037503,-0.154753,-1.005721,0.009081,-0.624618,-0.780838,...,-0.172593,-0.410390,0.056868,0.007147,0.410814,0.233064,0.047234,0.001865,0.033250,0.209758
std,365.251800,660.466253,0.004676,0.005630,0.005707,1.031666,2.642147,0.003466,0.071646,0.069515,...,0.353708,0.414356,0.231625,0.084252,0.492058,0.422848,0.212173,0.043146,0.179318,0.407199
min,0.000000,285.900000,0.008726,0.002392,0.012534,-2.125023,-5.999652,0.000051,-0.837559,-0.880751,...,-1.036083,-1.224658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,294.250000,3235.500000,0.025604,0.015975,0.034615,-0.975815,-2.849101,0.006964,-0.669787,-0.826073,...,-0.445837,-0.682809,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,595.000000,3600.000000,0.028527,0.017814,0.038156,-0.138377,-1.422820,0.009405,-0.625887,-0.796701,...,-0.180206,-0.468072,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,895.750000,3965.950000,0.031523,0.020183,0.041093,0.549419,1.228256,0.011456,-0.583248,-0.758747,...,0.108718,-0.193907,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1377.000000,10960.300000,0.046577,0.046411,0.053486,1.997825,4.592682,0.024836,-0.304926,-0.304926,...,0.773722,2.037502,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [46]:
def model_evaluation(y, y_pred):
    # Evaluate model performance
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y - y_pred))
    r2 = r2_score(y, y_pred)

    # the average squared difference between the predicted and actual values. A lower MSE indicates better performance.
    print('MSE:', mse)
    # square root of the MSE. A lower RMSE indicates better performance.
    print('RMSE:', rmse)
    # The average absolute difference between the predicted and actual values. 
    # It is less sensitive to outliers compared to the MSE and RMSE.
    print('MAE:', mae)
    # the dependent variable that is explained by the independent variables in the model. 
    # An R^2 value closer to 1 indicates better performance.
    print('R^2:', r2)

In [47]:
model_evaluation(y_train, lr_y_train_pred)

MSE: 420410.760170615
RMSE: 648.3909007463129
MAE: 472.47370842202224
R^2: 0.03712311525302914


In [48]:
model_evaluation(y_test, lr_y_test_pred)

MSE: 414545.5126250968
RMSE: 643.8520890896424
MAE: 470.92213054774635
R^2: 0.038656637129700244


In [ ]:
model_evaluation(X_train, y_train, X_test, y_test)

In [ ]:
# Create scatter plot
plt.scatter(lr_y_test_pred, y_test)
plt.xlabel('Predicted values')
plt.ylabel('Actual values')
plt.title('Scatter plot of predicted vs actual values')
plt.show()

In [ ]:
# Create residual plot lc_y_train_pred, 
residuals = y_test - lr_y_test_pred
# Create histogram of residuals
plt.hist(residuals, bins=20)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of residuals')
plt.show()

### The residuals are normally distributed

In [ ]:
# Create Q-Q plot of residuals
import scipy.stats as stats
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('residuals Q-Q plot')
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(2)

In [ ]:
poly.fit_transform(X_train)

# RidgeCV model

In [ ]:
# RidgeCV model
# Assigning the algorithm to the variable
rc = RidgeCV()

# Fitting of the model
rc.fit(X_train,y_train)

In [ ]:
rc_y_train_pred = rc.predict(X_train)
rc_y_test_pred = rc.predict(X_test)

In [ ]:
model_evaluation(X_train, y_train, X_test, y_test, rc)

In [ ]:
# Create scatter plot
plt.scatter(rc_y_test_pred, y_test)
plt.xlabel('Predicted values')
plt.ylabel('Actual values')
plt.title('Scatter plot of predicted vs actual values')
plt.show()

In [ ]:
# Create Q-Q plot of residuals
import scipy.stats as stats
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('residuals Q-Q plot')
plt.show()

# LassoCV model

In [ ]:
# LassoCV model
# Assigning the algorithm to the variable
lc = LassoCV()

# Fitting of the model
lc.fit(X_train,y_train)

In [ ]:
lc_y_train_pred = lc.predict(X_train)
lc_y_test_pred = lc.predict(X_test)

In [ ]:
# Create scatter plot
plt.scatter(lc_y_test_pred, y_test)
plt.xlabel('Predicted values')
plt.ylabel('Actual values')
plt.title('Scatter plot of predicted vs actual values')
plt.show()

In [ ]:
model_evaluation(X_train, y_train, X_test, y_test, lc)

In [ ]:
# Create residual plot lc_y_train_pred, 
residuals = y_test - lc_y_test_pred
# Create histogram of residuals
plt.hist(residuals, bins=20)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of residuals')
plt.show()

In [ ]:
# Create Q-Q plot of residuals
import scipy.stats as stats
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('residuals Q-Q plot')
plt.show()

In [ ]:
# Create box plot of residuals
plt.boxplot(residuals)
plt.ylabel('Residuals')
plt.title('Box plot of residuals')
plt.show()

In [ ]:
print(f"x_train shape {X.shape}")
print(f"y_train shape {y.shape}")
print(f"x_test shape {X.shape}")
print(f"y_test shape {y.shape}")

In [ ]:
plt.scatter(X_train['std_phen_savi2_2'], y_train, color = 'red')
plt.xlabel('std_phen_savi2_2')
plt.ylabel('Sitlav - Soybeans - Yield - KG/Ha')
plt.show()

In [ ]:
X.describe()

In [ ]:
plt.scatter(X_train['kurt_phen_savi2_1'], y_train, color = "blue")
#plt.plot(X_test['kurt_phen_savi2_1'], lr_y_test_pred, color = "red")
plt.xlabel('kurt_phen_savi2_1', color = "purple")
plt.ylabel('Sitlav - Soybeans - Yield - KG/Ha', color = "purple")
plt.title('Trained model plot', color = "purple")
plt.plot;